# Using custom containers with AI Platform Training

**Learning Objectives:**
1. Learn how to create a train and a validation split with Big Query
1. Learn how to wrap a machine learning model into a Docker container and train in on CAIP
1. Learn how to use the hyperparameter tunning engine on GCP to find the best hyperparameters
1. Learn how to deploy a trained machine learning model GCP as a rest API and query it

In this lab, you develop, package as a docker image, and run on **AI Platform Training** a training application that trains a multi-class classification model that predicts the type of forest cover from cartographic data. The [dataset](../../../datasets/covertype/README.md) used in the lab is based on **Covertype Data Set** from UCI Machine Learning Repository.

The training code uses `scikit-learn` for data pre-processing and modeling. The code has been instrumented using the `hypertune` package so it can be used with **AI Platform** hyperparameter tuning.


In [41]:
import json
import os
import numpy as np
import pandas as pd
import pickle
import uuid
import time
import tempfile

from googleapiclient import discovery
from googleapiclient import errors

from google.cloud import bigquery
from jinja2 import Template
from kfp.components import func_to_container_op
from typing import NamedTuple

from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import SGDClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer

## Configure environment settings

Set location paths, connections strings, and other environment settings. Make sure to update   `REGION`, and `ARTIFACT_STORE`  with the settings reflecting your lab environment. 

- `REGION` - the compute region for AI Platform Training and Prediction
- `ARTIFACT_STORE` - the GCS bucket created during installation of AI Platform Pipelines. The bucket name starts with the `hostedkfp-default-` prefix.

In [42]:
!gsutil ls

gs://artifacts.dmgcp-pkg-internal-poc-oct-04.appspot.com/
gs://datametica_internal/
gs://delete_keras_demo_wit/
gs://delete_mortgage_xgboost/
gs://dm_internal_demo/
gs://dmgcp-pkg-internal-poc-oct-04/
gs://dmgcp-pkg-internal-poc-oct-04-kubeflowpipelines-default/
gs://dmgcp-pkg-internal-poc-oct-04-mlengine/
gs://dmgcp-pkg-internal-poc-oct-04.appspot.com/
gs://dmgcp-pkg-internal-poc-oct-04_cloudbuild/
gs://europe-west1-my-composer-c04144a9-bucket/
gs://fastai-demo/
gs://hostedkfp-default-t0ns1asff6/
gs://image_classification_kyc/
gs://kyc_processing/
gs://qvc_femo_files/


In [43]:
REGION = 'us-central1'
ARTIFACT_STORE = 'gs://hostedkfp-default-t0ns1asff6'

PROJECT_ID = !(gcloud config get-value core/project)
PROJECT_ID = PROJECT_ID[0]
DATA_ROOT='{}/data'.format(ARTIFACT_STORE)
JOB_DIR_ROOT='{}/jobs'.format(ARTIFACT_STORE)
TRAINING_FILE_PATH='{}/{}/{}'.format(DATA_ROOT, 'training', 'dataset.csv')
VALIDATION_FILE_PATH='{}/{}/{}'.format(DATA_ROOT, 'validation', 'dataset.csv')

In [44]:
VALIDATION_FILE_PATH

'gs://hostedkfp-default-t0ns1asff6/data/validation/dataset.csv'

## Explore the Credit card dataset 

In [4]:
%%bigquery
SELECT *
FROM `MLOps_poc.credit_card_fraud`

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,NaN
1,7444.000000000,-1.732699558,-0.226281615,1.373927393,0.868954431,3.696451906,3.827377978,-0.897345356,0.537505032,1.490803114,...,-0.410690459,-0.366613948,-0.667907407,0.982553936,-0.317467795,-0.333377955,-0.498264124,-0.310378381,0E-9,0.0
2,8256.000000000,0.919153734,0.148087303,1.681714330,2.779928578,-0.105434428,2.032115669,-1.000826622,0.679995612,1.405340838,...,-0.099649840,0.249609434,0.228987663,-0.717434874,-0.095069397,0.027851205,0.086004683,0.017047698,0E-9,0.0
3,26373.000000000,-1.563484871,1.155879702,1.924913297,1.512296159,-0.447353620,1.280312133,-0.888981010,1.433368488,-0.470963337,...,0.170774257,0.454554592,-0.096082711,-0.276239147,-0.356016680,-0.210269702,-0.172889768,-0.015972867,0E-9,0.0
4,35377.000000000,1.126534327,0.289679104,1.537309696,2.706547133,-0.759367905,0.161174018,-0.529164795,0.141161868,0.046730494,...,-0.081740974,-0.059833686,0.041016588,0.398585954,0.312066335,-0.045035054,0.045694022,0.036644637,0E-9,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
284803,147501.000000000,-1.611877338,-0.408409923,-3.829761676,6.249461761,-3.360921609,1.147963582,1.858424853,0.474858318,-3.838398816,...,1.245582186,0.616382702,2.251438880,-0.066096392,0.538710327,0.541325493,-0.136243382,-0.009852256,996.270000000,1.0
284804,139951.000000000,-2.921944376,-0.228062381,-5.877288646,2.201883522,-1.935439873,0.631140872,-1.245106342,1.511348020,-1.899986657,...,1.441622275,0.895527949,1.385511128,-2.028024229,0.509130972,0.172642921,0.726781012,0.234513922,723.210000000,1.0
284805,131024.000000000,0.469749517,-1.237555470,-1.767340616,4.833490128,-0.268714559,-0.512759848,1.140148806,-0.341273000,-1.046350984,...,0.303905313,-0.647075252,-0.373013646,0.260801138,-0.496566412,-0.245972762,-0.117857669,0.144774181,723.210000000,1.0
284806,122608.000000000,-2.003459531,-7.159041717,-4.050976316,1.309579747,-2.058101588,-0.098620927,2.880082727,-0.727484047,1.460380551,...,1.244286775,-1.015232287,-1.800984866,0.657585627,-0.435617247,-0.894508922,-0.397557387,0.314261714,2125.870000000,1.0


## Create training and validation splits

Use BigQuery to sample training and validation splits and save them to GCS storage
### Create a training split

In [109]:
!bq query \
-n 0 \
--destination_table MLOps_poc.training \
--replace \
--use_legacy_sql=false \
'SELECT * \
FROM `MLOps_poc.credit_card_fraud` AS credit \
WHERE \
rand() < 0.8' 

Waiting on bqjob_r593b9a6198bde704_000001740eed79eb_1 ... (10s) Current status: DONE   


In [110]:
!bq extract \
--destination_format CSV \
MLOps_poc.training \
$TRAINING_FILE_PATH

Waiting on bqjob_r71f6c45f60046c86_000001740eedaf1b_1 ... (10s) Current status: DONE   


In [105]:
!bq query \
-n 0 \
--destination_table MLOps_poc.training1 \
--replace \
--use_legacy_sql=false \
'SELECT * \
FROM `MLOps_poc.credit_card_fraud` AS credit \
WHERE \
rand() < 0.7' 

Waiting on bqjob_r3829f7fc73fdb85b_000001740eeaf4da_1 ... (10s) Current status: DONE   


In [106]:
!bq extract \
--destination_format CSV \
MLOps_poc.training1 \
$TRAINING_FILE_PATH

Waiting on bqjob_r6659df1b4040b655_000001740eeb32dd_1 ... (6s) Current status: DONE   


### Create a validation split

In [111]:
!bq query \
-n 0 \
--destination_table MLOps_poc.validation \
--replace \
--use_legacy_sql=false \
'SELECT * \
FROM `MLOps_poc.credit_card_fraud` AS credit \
WHERE \
rand() < 0.2' 

Waiting on bqjob_r63a117c14d5604a_000001740eede488_1 ... (4s) Current status: DONE   


In [112]:
!bq extract \
--destination_format CSV \
MLOps_poc.validation \
$VALIDATION_FILE_PATH

Waiting on bqjob_r54bb20a8313a8acb_000001740eee037d_1 ... (3s) Current status: DONE   


In [84]:
df_train = pd.read_csv(TRAINING_FILE_PATH)
df_validation = pd.read_csv(VALIDATION_FILE_PATH)
print(df_train.shape)
print(df_validation.shape)

(227723, 31)
(57256, 31)


In [ ]:
df_train = pd.read_csv(TRAINING_FILE_PATH)
df_validation = pd.read_csv(VALIDATION_FILE_PATH)
print(df_train.shape)
print(df_validation.shape)

In [120]:
df_train.isnull().sum()

Time      0
V1        0
V2        0
V3        0
V4        0
V5        0
V6        0
V7        0
V8        0
V9        0
V10       0
V11       0
V12       0
V13       0
V14       0
V15       0
V16       0
V17       0
V18       0
V19       0
V20       0
V21       0
V22       0
V23       0
V24       0
V25       0
V26       0
V27       0
V28       0
Amount    0
Class     0
dtype: int64

## Develop a training application

### Configure the `sklearn` training pipeline.

The training pipeline preprocesses data by standardizing all numeric features using `sklearn.preprocessing.StandardScaler` and encoding all categorical features using `sklearn.preprocessing.OneHotEncoder`. It uses stochastic gradient descent linear classifier (`SGDClassifier`) for modeling.

In [86]:
from sklearn.ensemble import RandomForestClassifier
numeric_feature_indexes = slice(0, 30)
categorical_feature_indexes = slice(31)

preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numeric_feature_indexes), 
    ])

pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('classifier', RandomForestClassifier(max_depth=3))
])

### Convert all numeric features to `float64`

To avoid warning messages from `StandardScaler` all numeric features are converted to `float64`.

In [88]:
num_features_type_map = {feature: 'float64' for feature in df_train.columns[numeric_feature_indexes]}

df_train = df_train.astype(num_features_type_map)
df_validation = df_validation.astype(num_features_type_map)

### Run the pipeline locally.

In [90]:
X_train = df_train.drop('Class', axis=1)
y_train = df_train['Class']
X_validation = df_validation.drop('Class', axis=1)
y_validation = df_validation['Class']

pipeline.set_params(classifier__max_depth=3)
pipeline.fit(X_train, y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num', StandardScaler(),
                                                  slice(0, 30, None))])),
                ('classifier', RandomForestClassifier(max_depth=3))])

### Calculate the trained model's accuracy.

In [91]:
accuracy = pipeline.score(X_validation, y_validation)
print(accuracy)

0.9991616599133715


### Prepare the hyperparameter tuning application.
Since the training run on this dataset is computationally expensive you can benefit from running a distributed hyperparameter tuning job on AI Platform Training.

In [92]:
TRAINING_APP_FOLDER = 'training_app'
os.makedirs(TRAINING_APP_FOLDER, exist_ok=True)

### Write the tuning script. 

Notice the use of the `hypertune` package to report the `accuracy` optimization metric to AI Platform hyperparameter tuning service.

In [93]:
%%writefile {TRAINING_APP_FOLDER}/train.py

# Copyright 2019 Google Inc. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#            http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

import os
import subprocess
import sys

import fire
import pickle
import numpy as np
import pandas as pd

import hypertune
from sklearn.ensemble import RandomForestClassifier
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import SGDClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder


def train_evaluate(job_dir, training_dataset_path, validation_dataset_path, alpha, max_iter, hptune):
    
    df_train = pd.read_csv(training_dataset_path)
    df_validation = pd.read_csv(validation_dataset_path)

    if not hptune:
        df_train = pd.concat([df_train, df_validation])

    numeric_feature_indexes = slice(0, 30)


    preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numeric_feature_indexes)
    ])

    pipeline = Pipeline([
        ('preprocessor', preprocessor),
        ('classifier',RandomForestClassifier(max_depth=2))
    ])

    num_features_type_map = {feature: 'float64' for feature in df_train.columns[numeric_feature_indexes]}
    df_train = df_train.astype(num_features_type_map)
    df_validation = df_validation.astype(num_features_type_map) 

    print('Starting training: alpha={}, max_iter={}'.format(alpha, max_iter))
    X_train = df_train.drop('Class', axis=1)
    y_train = df_train['Class']

    pipeline.set_params(classifier__max_depth=alpha)
    pipeline.fit(X_train, y_train)

    if hptune:
        X_validation = df_validation.drop('Class', axis=1)
        y_validation = df_validation['Class']
        accuracy = pipeline.score(X_validation, y_validation)
        print('Model accuracy: {}'.format(accuracy))
        # Log it with hypertune
        hpt = hypertune.HyperTune()
        hpt.report_hyperparameter_tuning_metric(
          hyperparameter_metric_tag='accuracy',
          metric_value=accuracy
        )

    # Save the model
    if not hptune:
        model_filename = 'model.pkl'
        with open(model_filename, 'wb') as model_file:
            pickle.dump(pipeline, model_file)
        gcs_model_path = "{}/{}".format(job_dir, model_filename)
        subprocess.check_call(['gsutil', 'cp', model_filename, gcs_model_path], stderr=sys.stdout)
        print("Saved model in: {}".format(gcs_model_path)) 
    
if __name__ == "__main__":
    fire.Fire(train_evaluate)

Writing training_app/train.py


In [ ]:
!python  helper.py

Python 3.7.6 | packaged by conda-forge | (default, Jun  1 2020, 18:57:50) 
[GCC 7.5.0] on linux
Type "help", "copyright", "credits" or "license" for more information.
>>> 
Traceback (most recent call last):
  File "<stdin>", line 1, in <module>
KeyboardInterrupt
>>> 

### Package the script into a docker image.

Notice that we are installing specific versions of `scikit-learn` and `pandas` in the training image. This is done to make sure that the training runtime is aligned with the serving runtime. Later in the notebook you will deploy the model to AI Platform Prediction, using the 1.15 version of AI Platform Prediction runtime. 

Make sure to update the URI for the base image so that it points to your project's **Container Registry**.

In [94]:
%%writefile {TRAINING_APP_FOLDER}/Dockerfile

FROM gcr.io/deeplearning-platform-release/base-cpu
RUN pip install -U fire cloudml-hypertune scikit-learn==0.20.4 pandas==0.24.2
WORKDIR /app
COPY train.py .

ENTRYPOINT ["python", "train.py"]

Writing training_app/Dockerfile


### Build the docker image. 

You use **Cloud Build** to build the image and push it your project's **Container Registry**. As you use the remote cloud service to build the image, you don't need a local installation of Docker.

In [95]:
IMAGE_NAME='trainer_image'
IMAGE_TAG='latest'
IMAGE_URI='gcr.io/{}/{}:{}'.format(PROJECT_ID, IMAGE_NAME, IMAGE_TAG)

In [96]:
!gcloud builds submit --tag $IMAGE_URI $TRAINING_APP_FOLDER

Creating temporary tarball archive of 2 file(s) totalling 3.1 KiB before compression.
Uploading tarball of [training_app] to [gs://dmgcp-pkg-internal-poc-oct-04_cloudbuild/source/1597969559.67-a53216f0e0ad4001a65f6888a6b54051.tgz]
Created [https://cloudbuild.googleapis.com/v1/projects/dmgcp-pkg-internal-poc-oct-04/builds/226d8e4e-0620-4a17-bfb9-22035dee1bbf].
Logs are available at [https://console.cloud.google.com/cloud-build/builds/226d8e4e-0620-4a17-bfb9-22035dee1bbf?project=57394649199].
----------------------------- REMOTE BUILD OUTPUT ------------------------------
starting build "226d8e4e-0620-4a17-bfb9-22035dee1bbf"

FETCHSOURCE
Fetching storage object: gs://dmgcp-pkg-internal-poc-oct-04_cloudbuild/source/1597969559.67-a53216f0e0ad4001a65f6888a6b54051.tgz#1597969560142365
Copying gs://dmgcp-pkg-internal-poc-oct-04_cloudbuild/source/1597969559.67-a53216f0e0ad4001a65f6888a6b54051.tgz#1597969560142365...
/ [1 files][  1.5 KiB/  1.5 KiB]                                              

## Submit an AI Platform hyperparameter tuning job

### Create the hyperparameter configuration file. 
Recall that the training code uses `SGDClassifier`. The training application has been designed to accept two hyperparameters that control `SGDClassifier`:
- Max iterations
- Alpha

The below file configures AI Platform hypertuning to run up to 6 trials on up to three nodes and to choose from two discrete values of `max_iter` and the linear range betwee 0.00001 and 0.001 for `alpha`.

In [97]:
%%writefile {TRAINING_APP_FOLDER}/hptuning_config.yaml

# Copyright 2019 Google Inc. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#            http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

trainingInput:
  hyperparameters:
    goal: MAXIMIZE
    maxTrials: 4
    maxParallelTrials: 4
    hyperparameterMetricTag: accuracy
    enableTrialEarlyStopping: TRUE 
    params:
    - parameterName: max_iter
      type: DISCRETE
      discreteValues: [
          200,
          500
          ]
    - parameterName: alpha
      type: DISCRETE
      minValue:  2
      maxValue:  4
      scaleType: UNIT_LINEAR_SCALE

Writing training_app/hptuning_config.yaml


### Start the hyperparameter tuning job.

Use the `gcloud` command to start the hyperparameter tuning job.

In [98]:
JOB_NAME = "JOB_{}".format(time.strftime("%Y%m%d_%H%M%S"))
JOB_DIR = "{}/{}".format(JOB_DIR_ROOT, JOB_NAME)
SCALE_TIER = "BASIC"

!gcloud ai-platform jobs submit training $JOB_NAME \
--region=$REGION \
--job-dir=$JOB_DIR \
--master-image-uri=$IMAGE_URI \
--scale-tier=$SCALE_TIER \
--config $TRAINING_APP_FOLDER/hptuning_config.yaml \
-- \
--training_dataset_path=$TRAINING_FILE_PATH \
--validation_dataset_path=$VALIDATION_FILE_PATH \
--hptune

Job [JOB_20200821_003637] submitted successfully.
Your job is still active. You may view the status of your job with the command

  $ gcloud ai-platform jobs describe JOB_20200821_003637

or continue streaming the logs with the command

  $ gcloud ai-platform jobs stream-logs JOB_20200821_003637
jobId: JOB_20200821_003637
state: QUEUED


### Monitor the job.

You can monitor the job using GCP console or from within the notebook using `gcloud` commands.

In [99]:
!gcloud ai-platform jobs describe $JOB_NAME

createTime: '2020-08-21T00:36:39Z'
etag: jHjWWc5djVk=
jobId: JOB_20200821_003637
startTime: '2020-08-21T00:36:41Z'
state: RUNNING
trainingInput:
  args:
  - --training_dataset_path=gs://hostedkfp-default-t0ns1asff6/data/training/dataset.csv
  - --validation_dataset_path=gs://hostedkfp-default-t0ns1asff6/data/validation/dataset.csv
  - --hptune
  hyperparameters:
    enableTrialEarlyStopping: true
    goal: MAXIMIZE
    hyperparameterMetricTag: accuracy
    maxParallelTrials: 4
    maxTrials: 4
    params:
    - discreteValues:
      - 200.0
      - 500.0
      parameterName: max_iter
      type: DISCRETE
    - maxValue: 4.0
      minValue: 2.0
      parameterName: alpha
      scaleType: UNIT_LINEAR_SCALE
      type: DOUBLE
  jobDir: gs://hostedkfp-default-t0ns1asff6/jobs/JOB_20200821_003637
  masterConfig:
    imageUri: gcr.io/dmgcp-pkg-internal-poc-oct-04/trainer_image:latest
  region: us-central1
trainingOutput:
  hyperparameterMetricTag: accuracy
  isHyperparameterTuningJob: true

V

In [100]:
#!gcloud ai-platform jobs stream-logs $JOB_NAME

^C


Command killed by keyboard interrupt



### Retrieve HP-tuning results.

After the job completes you can review the results using GCP Console or programatically by calling the AI Platform Training REST POI-point.

In [101]:
ml = discovery.build('ml', 'v1')

job_id = 'projects/{}/jobs/{}'.format(PROJECT_ID, JOB_NAME)
request = ml.projects().jobs().get(name=job_id)

try:
    response = request.execute()
except errors.HttpError as err:
    print(err)
except:
    print("Unexpected error")
    
response

{'jobId': 'JOB_20200821_003637',
 'trainingInput': {'args': ['--training_dataset_path=gs://hostedkfp-default-t0ns1asff6/data/training/dataset.csv',
   '--validation_dataset_path=gs://hostedkfp-default-t0ns1asff6/data/validation/dataset.csv',
   '--hptune'],
  'hyperparameters': {'goal': 'MAXIMIZE',
   'params': [{'parameterName': 'max_iter',
     'type': 'DISCRETE',
     'discreteValues': [200, 500]},
    {'parameterName': 'alpha',
     'minValue': 2,
     'maxValue': 4,
     'type': 'DOUBLE',
     'scaleType': 'UNIT_LINEAR_SCALE'}],
   'maxTrials': 4,
   'maxParallelTrials': 4,
   'hyperparameterMetricTag': 'accuracy',
   'enableTrialEarlyStopping': True},
  'region': 'us-central1',
  'jobDir': 'gs://hostedkfp-default-t0ns1asff6/jobs/JOB_20200821_003637',
  'masterConfig': {'imageUri': 'gcr.io/dmgcp-pkg-internal-poc-oct-04/trainer_image:latest'}},
 'createTime': '2020-08-21T00:36:39Z',
 'startTime': '2020-08-21T00:36:41Z',
 'endTime': '2020-08-21T00:46:45Z',
 'state': 'SUCCEEDED',
 't

The returned run results are sorted by a value of the optimization metric. The best run is the first item on the returned list.

In [102]:
response['trainingOutput']['trials'][0]

{'trialId': '2',
 'hyperparameters': {'max_iter': '200', 'alpha': '3.4382479813356506'},
 'finalMetric': {'trainingStep': '1', 'objectiveValue': 0.9993188486796144},
 'startTime': '2020-08-21T00:37:18.016709243Z',
 'endTime': '2020-08-21T00:45:51Z',
 'state': 'SUCCEEDED'}

## Retrain the model with the best hyperparameters

You can now retrain the model using the best hyperparameters and using combined training and validation splits as a training dataset.

### Configure and run the training job

In [ ]:
alpha = response['trainingOutput']['trials'][0]['hyperparameters']['alpha']
max_iter = response['trainingOutput']['trials'][0]['hyperparameters']['max_iter']

In [ ]:
JOB_NAME = "JOB_{}".format(time.strftime("%Y%m%d_%H%M%S"))
JOB_DIR = "{}/{}".format(JOB_DIR_ROOT, JOB_NAME)
SCALE_TIER = "BASIC"

!gcloud ai-platform jobs submit training $JOB_NAME \
--region=$REGION \
--job-dir=$JOB_DIR \
--master-image-uri=$IMAGE_URI \
--scale-tier=$SCALE_TIER \
-- \
--training_dataset_path=$TRAINING_FILE_PATH \
--validation_dataset_path=$VALIDATION_FILE_PATH \
--alpha=$alpha \
--max_iter=$max_iter \
--nohptune

In [ ]:
!gcloud ai-platform jobs stream-logs $JOB_NAME

### Examine the training output

The training script saved the trained model as the 'model.pkl' in the `JOB_DIR` folder on GCS.

In [ ]:
!gsutil ls $JOB_DIR

## Deploy the model to AI Platform Prediction

### Create a model resource

In [ ]:
model_name = 'forest_cover_classifier'
labels = "task=classifier,domain=forestry"
filter = 'name:{}'.format(model_name)
models = !(gcloud ai-platform models list --filter={filter} --format='value(name)')

if not models:
    !gcloud ai-platform models create  $model_name \
    --regions=$REGION \
    --labels=$labels
else:
    print("Model: {} already exists.".format(models[0]))

### Create a model version

In [ ]:
model_version = 'v01'
filter = 'name:{}'.format(model_version)
versions = !(gcloud ai-platform versions list --model={model_name} --format='value(name)' --filter={filter})

if not versions:
    !gcloud ai-platform versions create {model_version} \
    --model={model_name} \
    --origin=$JOB_DIR \
    --runtime-version=1.15 \
    --framework=scikit-learn \
    --python-version=3.7
else:
    print("Model version: {} already exists.".format(versions[0]))

### Serve predictions
#### Prepare the input file with JSON formated instances.

In [ ]:
input_file = 'serving_instances.json'

with open(input_file, 'w') as f:
    for index, row in X_validation.head().iterrows():
        f.write(json.dumps(list(row.values)))
        f.write('\n')

In [ ]:
!cat $input_file

#### Invoke the model

In [ ]:
!gcloud ai-platform predict \
--model $model_name \
--version $model_version \
--json-instances $input_file

<font size=-1>Licensed under the Apache License, Version 2.0 (the \"License\");
you may not use this file except in compliance with the License.
You may obtain a copy of the License at [https://www.apache.org/licenses/LICENSE-2.0](https://www.apache.org/licenses/LICENSE-2.0)

Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an \"AS IS\" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.  See the License for the specific language governing permissions and limitations under the License.</font>